In [2]:
from compas.datastructures import Mesh
from compas.datastructures import subdivision as sd
from compas_plotters import MeshPlotter
from compas.geometry import midpoint_point_point, Vector
import random
from statistics import mean

from noise import pnoise1, snoise2, pnoise2, perlin
from utilities import draw_compas_mesh, export_obj_by_attribute
import mysubdivision as msd


def plot(m):
    plotter = MeshPlotter(m)
    plotter.draw_edges()
    plotter.draw_faces()
    plotter.show()

mesh = Mesh.from_json('ring_1.json')

draw_compas_mesh(mesh,size=10)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [3]:
def noises(bigY, bigX, octaves=1):
    freq = 16.0 * octaves

    noise_list = []

    for y in range(bigY):
        noise_list.append(pnoise2(-y / freq, y / freq, octaves))
    
    return noise_list

In [8]:
# Start subdividing

# weird setup
inner_faces = []
for inner_face in mesh.faces_where({'location': 'inner'}):
    inner_faces.append(inner_face)
    
bigY = len(inner_faces)

noise_values = noises(bigY, 3)
min_noise = min(noise_values)
max_noise = max(noise_values)

print(remap_noise[0:100])
print(noise_values[0:100])
    

[651.2481062569183, 597.9700109537937, 544.5345299689353, 490.15774517586016, 433.5835178948209, 374.5187330118337, 315.4617215157581, 262.6136139040011, 225.05270996025433, 212.2052828470149, 230.44921870263957, 280.15662396987517, 354.48153268280805, 440.72908949875705, 524.3397915388247, 594.6684172824607, 651.2481062569183, 702.2779780249016, 739.0654871282051, 747.2582495338027, 716.7835498566969, 646.2320870215962, 545.0321918522388, 432.18380018623645, 331.60155903442035, 265.41233803086436, 247.38907999256224, 278.81561311473064, 348.40670702110117, 436.6092452420818, 523.0412802794416, 594.5583705108893, 651.2481062569183, 705.9411675545543, 764.693257787734, 822.2241959330745, 868.9126946190157, 896.5234346869416, 902.0220135881851, 888.6575546324264, 864.3458044052503, 838.24922357544, 817.0095107750666, 802.1613002417579, 789.810709407003, 772.7955783608091, 744.4985193576233, 702.6395929356402, 651.2481062569183, 596.5629136082883, 538.1873182250089, 483.5664465425883, 447

In [22]:
mesh = Mesh.from_json('ring_1.json')


def move_vertices(mesh, fk, height):
    vertices = mesh.face_vertices(fk)
    vertices_coord = []
    
    face_location = mesh.face_centroid(fk)

    direction = Vector.from_start_end(face_location, [0,0,0])
    direction.unitize()
    direction = [i * height * pnoise1(fk, 2) for i in direction]

    for v in vertices:
        vertices_coord = mesh.vertex_coordinates(v)

        new_loc = Vector.sum_vectors([direction, vertices_coord])

        mesh.vertex[v]['x'] = new_loc[0]
        mesh.vertex[v]['y'] = new_loc[1]
        mesh.vertex[v]['z'] = new_loc[2]
        


def subdivide_by_ftype(mesh):
    fkeys = list(mesh.faces())
    for fk in fkeys:
        location = mesh.get_face_attribute(fk, 'location')
    
        # OUTER RING MAHIRO 
        if location == 'outer':
            new_keys = msd.taper_face(mesh, fk, height=.5, ratio=0.1)
            
            areas = []
            
            for new_key in new_keys:
                areas.append(mesh.face_area(new_key))
                
            for new_key in new_keys:
                if mesh.face_area(new_key) > mean(areas):
                    mesh.set_face_attribute(new_key, 'location', 'hub')
                else:
                    mesh.set_face_attribute(new_key, 'location', 'circulation')
        
        elif location == 'circulation':
            new_keys = msd.pyramid_face(mesh, fk, height=1)
            
            if random.random()<0.2:
                mesh.set_faces_attribute(new_keys, 'location', 'depot')
            else:
                mesh.set_faces_attribute(new_keys, 'location', 'conduits')
        
        elif location == 'conduits':
            new_key = msd.segment_face(mesh, fk, num=10, start_index=0)
            mesh.set_face_attribute(new_key, 'location', 'line')
             
        elif location == 'hub':
            new_keys = msd.segment_face(mesh, fk, num=10, start_index=0)
            mesh.set_faces_attribute(new_keys, 'location', 'part_hub')
        
        elif location == 'construction':
            pass
        
        # INNER RING TETOV
        elif location == 'inner':
            mesh.set_face_attribute(fk, 'location', 'inner_segmented')
            
        elif location == 'inner_segmented':
            
            noise_value = perlin.randint(-1,1)
            
            new_keys = msd.pyramid_face(mesh, fk, height=noise_value)
            
            new_value = (((noise_value - min_noise) * (bigY) - 0) / (max_noise - min_noise)) + 0
            
            if fk > noise_value:
                mesh.set_faces_attribute(new_keys, 'location', 'mountain_region')
            else:
                mesh.set_faces_attribute(new_keys, 'location', 'sea')
        
        elif location == 'mountain_region':
            move_vertices(mesh, fk, 2.5)
            
            new_keys = msd.segment_face(mesh, fk, num=4)
            
            mesh.set_face_attribute(new_keys, 'location', 'mountain')
            
        elif location == 'mountain':
            new_keys = msd.taper_face(mesh, fk, height=pnoise1(fk, 2)*10, ratio=perlin.randint(0,1))
            mesh.set_faces_attribute(new_keys, 'location', 'peaks')
        
        elif location == 'sea':
            move_vertices(mesh, fk, -2.5)
            
            mesh.set_face_attribute(fk, 'location', 'sea_grave')

        elif location == 'sea_grave':
            new_keys = msd.taper_face(mesh, fk, height=pnoise1(fk, 2)*-10, ratio=perlin.randint(0,1))
            mesh.set_faces_attribute(new_keys, 'location', 'sea_bottom')
             
        # SIDES
        elif location == 'sides':
            pass
        
        else:
            pass
Mesh.face_vertices
for _ in range(5):
    subdivide_by_ftype(mesh)

In [23]:
draw_compas_mesh(mesh)

export_obj_by_attribute('inner.obj', mesh, 'location')